In [ ]:
import numpy as np
from tqdm import tqdm
import keras
from keras import Input, Model
from keras.layers import Layer
from keras import backend as K
from keras import layers
from keras.preprocessing import timeseries_dataset_from_array

# from sklearn.metrics import accuracy_score, classification_report
# from keras.callbacks import ModelCheckpoint

from LobTransformer import TransformerBlock

In [73]:
from datetime import datetime

In [88]:
a=datetime.now()

In [96]:
b=datetime.now()

In [98]:
(b-a).seconds

60

In [121]:
# Load data
a={}
# download FI2010 dataset from 
# https://etsin.fairdata.fi/dataset/73eb48d7-4dbc-4a10-a52a-da745b47a649
FI2010_DIR = r'D:\WORKS\translob\dataset\BenchmarkDatasets'

def gen_data(data, horizon):
    x = data[:40, :].T  # 40 == 10 price + volume asks + 10 price + volume bids
    # FIXME: delete .T
    y = data[-5 + horizon, :].T  # 5
    return x[:-1], (y[1:] - 1).astype(np.int32)  # shift y by 1


def load_dataset(horizon):
    global a
    a.update({'start':datetime.now()})
    
    dec_data = np.loadtxt(
        f'{FI2010_DIR}/NoAuction/1.NoAuction_Zscore/NoAuction_Zscore_Training/Train_Dst_NoAuction_ZScore_CF_7.txt'
    )
    a.update({'train':datetime.now()})
    
    dec_train = dec_data[:, :int(np.floor(dec_data.shape[1] * 0.8))]
    dec_val = dec_data[:, int(np.floor(dec_data.shape[1] * 0.8)):]
    
    a.update({'train_eval':datetime.now()})
    
    dec_test1 = np.loadtxt(
        f'{FI2010_DIR}/NoAuction/1.NoAuction_Zscore/NoAuction_Zscore_Testing/Test_Dst_NoAuction_ZScore_CF_7.txt'
    )
    
    a.update({'test1':datetime.now()})
    
    dec_test2 = np.loadtxt(
        f'{FI2010_DIR}/NoAuction/1.NoAuction_Zscore/NoAuction_Zscore_Testing/Test_Dst_NoAuction_ZScore_CF_8.txt'
    )
    
    a.update({'test2':datetime.now()})
    
    dec_test3 = np.loadtxt(
        f'{FI2010_DIR}/NoAuction/1.NoAuction_Zscore/NoAuction_Zscore_Testing/Test_Dst_NoAuction_ZScore_CF_9.txt'
    )
    a.update({'test3':datetime.now()})
    
    dec_test = np.hstack((dec_test1, dec_test2, dec_test3))
    
    a.update({'stack':datetime.now()})
    
    result = (
        gen_data(dec_train,horizon), 
        gen_data(dec_val,horizon), 
        gen_data(dec_test, horizon)) #yapf: disable
    
    a.update({'result':datetime.now()})
    return result

In [122]:
def gen_dif(a):
    keys= list(a.keys())
    result={ 
        keys[i]:(a[keys[i]]-a[keys[i-1]]).total_seconds()
        for i in range(1,len(keys))
    } #yapf:disable
    return result

In [123]:
(x_train, y_train), (x_val, y_val), (x_test, y_test) = load_dataset(horizon=4)

In [124]:
gen_dif(a)

{'train': 9.116525,
 'train_eval': 0.0,
 'test1': 1.962616,
 'test2': 1.857686,
 'test3': 1.187247,
 'stack': 0.066204,
 'result': 0.001999}

In [ ]:
# test of fi2020 module from start
# a = np.array([
#     2615, 353, 2618, 200, 2619, 164, 2620, 532, 2621, 151, 2623, 837, 2625,
#     150, 2626, 787, 2629, 146, 2633, 311, 2606, 326, 2604, 682, 2602, 786,
#     2600, 893, 2599, 159, 2595, 100, 2593, 143, 2591, 134, 2588, 123, 2579, 128
# ])
# x_train[0]*a+a

In [ ]:
class PositionalEncodingLayer(Layer):

    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    def call(self, x, *args, **kwargs):
        steps, d_model = x.get_shape()[-2:]
        global A 
        A= x,steps,d_model
        ps = np.zeros([steps, 1], dtype=K.floatx())
        for step in range(steps):
            ps[step, :] = [(2 / (steps - 1)) * step - 1]

        ps_expand = K.expand_dims(K.constant(ps), axis=0)
        ps_tiled = K.tile(ps_expand, [K.shape(x)[0], 1, 1])

        x = K.concatenate([x, ps_tiled], axis=-1)
        return x


def eval(model, X_test, y_test, **kwargs):
    ts = TimeseriesGenerator(X_test,
                             y_test,
                             kwargs.get('sequence_length', 100),
                             batch_size=32,
                             shuffle=False)
    y_true = np.concatenate([y for x, y in ts])
    y_pred = np.argmax(model.predict(ts), -1)
    print(classification_report(y_true, y_pred))
    return classification_report(y_true, y_pred,
                                 output_dict=True)['weighted avg']['f1-score']


In [ ]:
class OptionsClass:
    """
    make only lover case parametrs and not start with _
    All this methods (exept __call__) only for beauty representation :)
    """
    _unpack_dicts_ = False

    def __call__(self, **kwargs: dict):
        """
        Set up parametrs
        """
        for key, value in kwargs.items():
            self.__setattr__(key, value)

    def __repr__(self):
        key_list = []
        for composite_key, value in self.Options.items():
            add_str = f'{composite_key}: '

            if isinstance(value, np.ndarray | range):
                value_str = f'[{value[0]}, {value[1]} .. {value[-2]}, {value[-1]}]; len = {len(value)}'

            else:
                value_str = f'{value}'

            key_list.append(add_str + value_str)
        return str('\n'.join(key_list))

    @property
    def Options(self):
        """
        Containing options dict
        """
        return {
            compound_key.strip(): value
            for value, compound_key in self._recursion_view()
        }

    def _all_options(self):
        data_filter = lambda x: (x[0] != '_') and x[0].islower()
        options = list(filter(data_filter, self.__dir__()))
        return {option: self.__getattribute__(option) for option in options}

    def _recursion_view(self, composite_key=''):
        """
        Need for recursion representation containing options
        """
        dilimiter = '  '
        if isinstance(self, OptionsClass):
            for key in self._all_options():
                for inner_key in OptionsClass._recursion_view(
                        self.__getattribute__(key),
                        str(composite_key) + dilimiter + str(key),
                ):
                    yield inner_key

        # Dicts store as dicts and include in composite key
        # (to unpack dicts - uncomment below):

        elif isinstance(self, dict) and OptionsClass._unpack_dicts_:
            for key in self:
                for inner_key in OptionsClass._recursion_view(
                        self[key],
                        str(composite_key) + dilimiter + str(key),
                ):
                    yield inner_key

        else:
            yield (self, composite_key)


In [ ]:
 # def __str__(self):
    #     key_list = []
    #     options= {
    #         compound_key.strip(): value
    #         for value, compound_key in self._recursion_view()
    #     }
    #     for composite_key, value in options.items():
    #         add_str = f'{composite_key}: '

    #         if isinstance(value, np.ndarray | range):
    #             value_str = f'[{value[0]}, {value[1]} .. {value[-2]}, {value[-1]}]; len = {len(value)}'

    #         else:
    #             value_str = f'{value}'

    #         key_list.append(add_str + value_str)
    #     return str('\n'.join(key_list))
        
    # def _recursion_view(self, composite_key=''):
        # """
        # Need for recursion representation containing options
        # """
        # dilimiter = '  '
        # if isinstance(self, OptionsClass):
        #     for key in self._get_all_options():
        #         for inner_key in OptionsClass._recursion_view(
        #                 self.__getattribute__(key),
        #                 str(composite_key) + dilimiter + str(key),
        #         ):
        #             yield inner_key

        # # Dicts store as dicts and include in composite key
        # # (to unpack dicts - uncomment below):

        # elif isinstance(self, dict) and OptionsClass._unpack_dicts_:
        #     for key in self:
        #         for inner_key in OptionsClass._recursion_view(
        #                 self[key],
        #                 str(composite_key) + dilimiter + str(key),
        #         ):
        #             yield inner_key

        # else:
        #     yield (self, composite_key)

In [ ]:
class DataClass:
    """
    make only lover case parametrs and not start with _
    All this methods (exept __call__) only for beauty representation :)
    """
    _data_filter = lambda x: (x[0] != '_') and ('Data' not in x)

    def __new__(
        cls,
        target_dict: dict = None,
        name: str='',
    ):
        if target_dict is not None:
            result =DataClass()
            return result._rec_build_(name, target_dict)
        return super().__new__(cls)
    
    def __call__(self, **kwargs: dict):
        """
        Set up parametrs
        """
        for key, value in kwargs.items():
            self.__setattr__(key, value)

    def __repr__(self) -> str:
        """
        Representation of options
        """
        return DataClass._rec_print_(self.Data)

    @property
    def Data(self):
        """
        Containing options dict
        """
        return self._rec_dict_()

    def _get_all_options(self):
        # Add except fields

        options = list(filter(DataClass._data_filter, self.__dir__()))
        return options

    def _rec_dict_(self, self_name=None):
        if not isinstance(self, DataClass):
            return {self_name: self}

        result = {}
        for option in self._get_all_options():

            inner_dict = DataClass._rec_dict_(
                self.__getattribute__(option),
                option,
            )
            result.update(inner_dict)

        if self_name is None:
            return result
        else:
            return {self_name: result}

    @staticmethod
    def _rec_print_(target, margin: str = ''):
        if not isinstance(target, dict):
            return f'{target}'
        result = margin

        for key, value in target.items():
            inner_string = DataClass._rec_print_(
                value,
                margin + ' ' * 4,
            )
            result += '\n' + margin + f'{key}: {inner_string}'

        if margin == '':
            return result[1:]
        else:
            return result

    def _rec_build_(self, self_name: str, target):
        if not isinstance(target, dict):
            self.__setattr__(self_name, target)
            return
        
        result = DataClass()
        self.__setattr__(self_name, result)
        

        for name, inner_target in target.items():
            inner_result = result._rec_build_(
                name,
                inner_target,
            )
            if inner_result is not None:
                self.__setattr__(self_name, inner_result)
        return result


In [ ]:
# Model parametrs
seq_len = 100

class CN_pars(DataClass):
    n_filters=14
    dilation_steps=4  # dilation = 2**dilation_step

class AN_pars(DataClass):
    attention_heads = 1
    blocks = 2
    share_weights = True

class FF_pars(DataClass):
    dropout_rate = 0.1

class Optimaser_pars(DataClass):
    lr = 0.0001
    adam_beta1 = 0.9
    adam_beta2 = 0.999

class Trainig_pars(DataClass):
    batch_size = 512
    epochs = 150
    
class Full_pars(DataClass):
    cn= CN_pars()
    an = AN_pars()
    ff=FF_pars()
    optimazer=Optimaser_pars()
    training=Trainig_pars()
    
pars= Full_pars() 

In [ ]:
# # Model parametrs
# seq_len = 100

# cn_pars1 = dict(
#     n_filters=14,
#     dilation_steps=4,
# )  # dilation = 2**dilation_step

# # AN
# attention_heads = 1
# blocks = 2
# share_weights = True

# # FF
# dropout_rate = 0.1

# # Optimizer
# lr = 0.0001
# adam_beta1 = 0.9
# adam_beta2 = 0.999

# # Training
# batch_size = 512
# epochs = 150

In [ ]:
# Model

# Input
inputs = Input(shape=(seq_len, 40))
x = inputs

# CN
dilation_steps = [2**dilation for dilation in range(pars.cn.dilation_steps + 1)]
for dilation in dilation_steps:
    x = layers.Conv1D(
        filters=pars.cn.n_filters,
        kernel_size=2,
        dilation_rate=dilation,
        activation='relu',
        padding='causal',
    )(x)
# Normalisation
norm = layers.LayerNormalization()(x)
x=norm

# Positional encoding
pos = PositionalEncodingLayer()(x)
x = pos

# Transformers
tb = TransformerBlock(
    'transformer_block',
    pars.an.attention_heads,
    True,
)
for block in range(pars.an.blocks):
    if pars.an.share_weights:
        x = tb(x)
    else:
        x = TransformerBlock(
            f'transformer_block_{block}',
            pars.an.attention_heads,
            True,
        )(x)

# FFN
x = layers.Flatten()(x)
x = layers.Dense(64,
                 activation='relu',
                 kernel_regularizer='l2',
                 kernel_initializer='glorot_uniform')(x)
x = layers.Dropout(pars.ff.dropout_rate)(x)
out = layers.Dense(3, activation='softmax')(x)
model = Model(inputs=inputs, outputs=out)

# CompiLe
model.summary()

model.compile(
    keras.optimizers.Adam(
        learning_rate=pars.optimazer.lr,
        beta_1=pars.optimazer.adam_beta1,
        beta_2=pars.optimazer.adam_beta2,
        name="Adam",
    ),
    loss=keras.losses.SparseCategoricalCrossentropy(),
    metrics=['sparse_categorical_accuracy'],
)
print(
    f'Train x: {str(x_train.shape): <15} - y: {y_train.shape}',
    f'Val   x: {str(x_val.shape): <15} - y: {y_val.shape}',
    sep='\n',
)

In [ ]:
def set_shape(value_x,value_y):
    value_x.set_shape((None,100,40))
    # value_y.set_shape((None,))
    return value_x,value_y

ds_train = timeseries_dataset_from_array(
    data=x_train,
    targets=y_train,
    batch_size=pars.training.batch_size,
    sequence_length=seq_len,
    shuffle=True,
)
ds_train= ds_train.map(set_shape)

ds_val = timeseries_dataset_from_array(
    data=x_val,
    targets=y_val,
    batch_size=pars.training.batch_size,
    sequence_length=seq_len,
    shuffle=True,
)
ds_val= ds_val.map(set_shape)


In [ ]:
model.fit(
    ds_train,
    epochs=pars.training.epochs,
    validation_data=ds_val,
)